In [ ]:
 !apt-get install -y -qq software-properties-common python-software-properties module-init-tools

#!add-apt-repository -y ppa:alessandro-strada/ppa  # 2>&1 > /dev/null

#!echo "PPA ADDED"
#!apt-get update  # 2>&1 > /dev/null
#!echo "REPO UPDATED"

!apt install software-properties-common dirmngr gnupg-agent
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys AD5F235DF639B041
!echo 'deb http://ppa.launchpad.net/alessandro-strada/ppa/ubuntu xenial main' | tee /etc/apt/sources.list.d/alessandro-strada-ubuntu-ppa.list >/dev/null
!apt-get update
!apt-get install -y -qq google-drive-ocamlfuse fuse
!echo "install done"

from google.colab import auth

auth.authenticate_user()



from oauth2client.client import GoogleCredentials

creds = GoogleCredentials.get_application_default()

import getpass

!google-drive-ocamlfuse -headless -id {creds.client_id} -secret {creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id {creds.client_id} -secret {creds.client_secret}
#vcode = getpass.getpass()


!mkdir -p drive

!google-drive-ocamlfuse drive



print('Files in Drive:')

!ls drive/



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import scipy
import cv2
import numpy as np

X_train=[]
y_train=[]
flg = False
#!ls "/content/drive/Train"
if (flg == False):
  path="/content/drive/My Drive/Train/"
  s=os.listdir("/content/drive/My Drive/Train")
  print(len(s))
  for i in range(len(s)):
      direction=s[i]
      tr=os.path.join(path,direction)
      gh=cv2.imread(tr)
      X_train.append(gh)
      k=str.split(direction,'_')
      if (k[0] == '100'):
          y_train.append(4)
      elif (k[0] == '5'):
          y_train.append(0)
      elif (k[0]=='10'):
          y_train.append(1)
      elif(k[0]=='25'):
          y_train.append(2)
      elif(k[0]=='50'):
          y_train.append(3)
      print(i)
  y_train = np.array(y_train)
  np.save(y_train , '/content/drive/MyDrive/temp.npy')
else:
  y_train = np.load('/content/drive/MyDrive/y_train.npy')


X_test=[]
X_test_name=[]
path="/content/drive/My Drive/Test/"
s=os.listdir("/content/drive/My Drive/Test")
for i in range(len(s)):
  direction=s[i]
  tir=os.path.join(path,direction)
  X_test.append(cv2.imread(tir))
  X_test_name.append(direction)
  print(i)





In [ ]:
import numpy as np
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

from keras.utils import np_utils

np.random.seed(2019)

X_train=np.array(X_train)
print(X_train.shape)
X_test=np.array(X_test)
trainX = np.reshape(X_train, [2352, 80, 80,3])
testX = np.reshape(X_test, [600, 80, 80,3])

print(trainX.shape)
print(testX.shape)

trainX_normal = (trainX/255.)
testX_normal = (testX/255.)


trainX_normal = trainX_normal.astype('float32')
testX_normal = testX_normal.astype('float32')


Y_train = np_utils.to_categorical(y_train, 5)


In [ ]:

input_shape = (80, 80,3)
model = Sequential()
model.add(Dropout(0.2, input_shape=input_shape))
model.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu'))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))




In [ ]:
#decay_rate = 0.01
#sgd = keras.optimizers.sgd(lr=0.1, momentum=0.9, decay=decay_rate)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(trainX_normal, Y_train,
          batch_size=46,
          epochs=10,
          verbose=1,
          shuffle=True,
          validation_split=0.1)  #validation_data=#(trainX_normal, Y_train))

#score = model.evaluate(testX_normal, Y_test, verbose=1)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])



W0709 17:58:39.348661 140087265470336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0709 17:58:39.382738 140087265470336 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0709 17:58:39.655951 140087265470336 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2000 samples, validate on 223 samples
Epoch 1/10
2000/2000 [==============================] - 6s 3ms/step - loss: 1.0794 - acc: 0.5755 - val_loss: 0.2320 - val_acc: 0.9372
Epoch 2/10
2000/2000 [==============================] - 1s 311us/step - loss: 0.6149 - acc: 0.8110 - val_loss: 0.2252 - val_acc: 0.9462
Epoch 3/10
2000/2000 [==============================] - 1s 309us/step - loss: 0.4885 - acc: 0.8560 - val_loss: 0.0844 - val_acc: 0.9910
Epoch 4/10
2000/2000 [==============================] - 1s 309us/step - loss: 0.4498 - acc: 0.8680 - val_loss: 0.0865 - val_acc: 0.9821
Epoch 5/10
2000/2000 [==============================] - 1s 295us/step - loss: 0.3686 - acc: 0.8950 - val_loss: 0.0736 - val_acc: 0.9731
Epoch 6/10
2000/2000 [==============================] - 1s 303us/step - loss: 0.3058 - acc: 0.9160 - val_loss: 0.1277 - val_acc: 0.9596
Epoch 7/10
2000/2000 [==============================] - 1s 296us/step - loss: 0.2863 - acc: 0.9170 - val_loss: 0.0520 - val_acc: 0.9821
Epo

In [ ]:
score = model.evaluate(trainX_normal, Y_train, verbose=1)
print('Train loss:', score[0])
print('Train accuracy:', score[1])
predicted_classes=model.predict(testX_normal)
print(predicted_classes[0])
print(predicted_classes.shape)
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)
#score = model.evaluate(testX_normal, Y_test, verbose=1)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])
#print(predicted_classes)


2223/2223 [==============================] - 0s 170us/step
Train loss: 0.3955470160440824
Train accuracy: 0.8353576248849345
[1.9577181e-03 9.9073762e-01 7.1797096e-03 1.2392621e-04 1.1201350e-06]
(600, 5)


In [ ]:
import csv

with open('/content/drive/My Drive/sample1.csv','w',newline='') as f:
  H=csv.writer(f)
  H.writerow(['ID','Predicted'])
  for i in range(600):
    k=str.split(X_test_name[i],'.')
    if(predicted_classes[i]==0):
      Target=[int(k[0])+1,5]
      H.writerow(Target)
    elif(predicted_classes[i]==1):
      Target=[int(k[0])+1,10]
      H.writerow(Target)
    elif(predicted_classes[i]==2):
      Target=[int(k[0])+1,25]
      H.writerow(Target)
    elif(predicted_classes[i]==3):
      Target=[int(k[0])+1,50]
      H.writerow(Target)
    elif(predicted_classes[i]==4):
      Target=[int(k[0])+1,100]
      H.writerow(Target)

f.close()